In [1]:
from pynvml import *
from os import listdir
from os.path import isfile, join
import numpy as np
import tifffile

In [2]:
def get_files(path,normalise=False,remove_txt=False):
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

    if remove_txt:
        onlyfiles = [val for val in onlyfiles if not val.endswith(".txt")]

    onlyfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    #if num_imgs > len(onlyfiles): num_imgs = len(onlyfiles)
    files = [np.squeeze(tifffile.imread(path +  onlyfiles[i])) for i in range(len(onlyfiles))]
    
    if normalise:
        mean = np.mean(files)
        std = np.std(files)
        standardized_files = [(image-mean)/std for image in files]
        return standardized_files
    
    return files
    
def get_data(path, set='01',normalise_images=True):

    if len(set) == 2: #set 01 or set 02
        images_path = path + set + '/'
        images = get_files(images_path,normalise=normalise_images)
        masks_path = path + set + '_GT/TRA/'
        masks = get_files(masks_path,remove_txt=True)
    elif set == '0102': #both sets
        images_path = path + '01/'
        images_01 = get_files(images_path,normalise=normalise_images)
        images_path = path + '02/'
        images_02 = get_files(images_path,normalise=normalise_images)
        images = images_01 + images_02

        masks_path = path + '01_GT/TRA/'
        masks_01 = get_files(masks_path,remove_txt=True)
        masks_path = path + '02_GT/TRA/'
        masks_02 = get_files(masks_path,remove_txt=True)
        masks = masks_01 + masks_02
    else:
        images = []
        masks = []

    return images, masks


images, masks = get_data("c:\\Users\\rz200\\Documents\\development\\distillCellSegTrack\\" + 'datasets/Fluo-N2DH-GOWT1/', set='0102', normalise_images=False)

In [3]:
import torch
print(torch.cuda.memory_allocated(0))

0


c:\Users\rz200\AppData\Local\anaconda3\envs\cellprob\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def split_image(img):
    
    # Split the input array into smaller arrays of size 256x256
    sub_images = []
    for i in range(0, img.shape[0], 256):
        for j in range(0, img.shape[1], 256):
            sub_img = img[i:i+256, j:j+256]
            sub_images.append(sub_img)
            
    return sub_images

In [14]:
#import cellpose model
from cellpose import models
from cellpose import core
import torch
import time
from u_net import UNet

nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
memory_before = torch.cuda.memory_allocated(0)
memory_used_list = []
times_taken_list = []
model = UNet()
model.to("cuda:0")
#model.load_state_dict(torch.load("c:\\Users\\rz200\\Documents\\development\\distillCellSegTrack\\segmentation\\train_dir\\models\\distillation_model_GOWT1_5"))
model.load_state_dict(torch.load("c:\\Users\\rz200\\Documents\\development\\distillCellSegTrack\\segmentation\\train_dir\\models\\unet_no_distillation_GOWT1"))

model.eval()
for i in range(len(images)):
    image = images[i]

    
    
    #model = models.CellposeModel(gpu=core.use_gpu(), device=torch.device("cuda:0"), pretrained_model="c:\\Users\\rz200\\Documents\\development\\distillCellSegTrack\\segmentation\\train_dir\\models\\cellpose_trained_model_SIM_4")
    
    splitted = split_image(image)
    torched_splitted = [torch.from_numpy(split).to('cuda:0') for split in splitted]
    start = time.time()
    for split in torched_splitted:
        #split = torch.from_numpy(split)
        split = split.unsqueeze(0)
        split = split.unsqueeze(0)
        prediction = model(split)
        
        del split
    times_taken_list.append(time.time() - start)
    #model = model.eval()

    

    memory_used = torch.cuda.memory_allocated(0)
    memory_used_list.append(memory_used)

In [15]:
from statistics import mean, stdev

print(mean(memory_used_list))
print(stdev(memory_used_list))

print(mean(memory_used_list)/1024**2)
print(stdev(memory_used_list)/1024**2)

46610432
0.0
44.451171875
0.0


In [16]:
print(mean(times_taken_list)*1000)
print(stdev(times_taken_list)*1000)

15.278248683266018
1.6860235199548168


In [12]:
print(times_taken_list)

[0.016002893447875977, 0.01601409912109375, 0.01501011848449707, 0.015023231506347656, 0.015003442764282227, 0.015009403228759766, 0.01512765884399414, 0.01501321792602539, 0.015014171600341797, 0.01600337028503418, 0.015015363693237305, 0.014014482498168945, 0.015003442764282227, 0.015016794204711914, 0.01501607894897461, 0.01500248908996582, 0.015003681182861328, 0.015165090560913086, 0.0150146484375, 0.016003131866455078, 0.016002416610717773, 0.015013694763183594, 0.015748023986816406, 0.015999794006347656, 0.01500248908996582, 0.015003204345703125, 0.01500558853149414, 0.015007257461547852, 0.015006303787231445, 0.01452779769897461, 0.015015840530395508, 0.015006303787231445, 0.015003204345703125, 0.015003204345703125, 0.016002893447875977, 0.01400613784790039, 0.015002727508544922, 0.015004396438598633, 0.015006065368652344, 0.015002012252807617, 0.015999794006347656, 0.015235185623168945, 0.015012979507446289, 0.01500082015991211, 0.01500844955444336, 0.016002893447875977, 0.015

In [8]:
print(times_taken_list)

[2.143925189971924, 0.033075809478759766, 0.03326773643493652, 0.03314495086669922, 0.032013893127441406, 0.032004594802856445, 0.03224039077758789, 0.03199458122253418, 0.03301095962524414, 0.033006906509399414, 0.031023263931274414, 0.03099989891052246, 0.03200054168701172, 0.0299990177154541, 0.03140425682067871, 0.0321660041809082, 0.0319981575012207, 0.031007766723632812, 0.0300137996673584, 0.031226158142089844, 0.030504703521728516, 0.031362295150756836, 0.030997514724731445, 0.03201031684875488, 0.030918598175048828, 0.031295061111450195, 0.03125929832458496, 0.031000137329101562, 0.03199934959411621, 0.03200268745422363, 0.031009674072265625, 0.03299546241760254, 0.03400301933288574, 0.03262925148010254, 0.03101325035095215, 0.03399038314819336, 0.04300260543823242, 0.03300023078918457, 0.03101038932800293, 0.03206634521484375, 0.03202033042907715, 0.03000807762145996, 0.031008243560791016, 0.03000807762145996, 0.030007362365722656, 0.032000064849853516, 0.029999494552612305, 

In [29]:
memory_standard = []
for mem in memory_used_list:
    memory_standard.append(mem-memory_before)
print(mean(memory_standard))

-1009254.4


In [16]:
!nvidia-smi

Fri May 12 15:30:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.01       Driver Version: 516.01       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000   WDDM  | 00000000:01:00.0  On |                  Off |
| 41%   34C    P8    15W / 140W |   2402MiB / 16376MiB |     32%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [99]:
torch.cuda.memory_allocated(0)

26501120

In [115]:
apples_before = 10

appleds_now = 11

apples_taken = apples_before - take_apple
print(apples_after)

9


In [73]:
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)

print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

total    : 17171480576
free     : 14666846208
used     : 2504634368
